# Google Drive

In [1]:
####################################
#
#  ADD THIS TO EVERY COLAB FILE!
#
####################################

from google.colab import drive
drive.mount('/content/drive')

import drive.Shareddrives.GPTJ.project.settings as settings

PATH_PROJECT = settings.PATH_PROJECT
PATH_DATA = settings.PATH_DATA

Mounted at /content/drive


In [2]:
! cd $PATH_PROJECT && pip install -q -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.0/144.0 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
!nvidia-smi

# Standard -> Tesla T4
# Premium -> Tesla P100-PCIE-16GB

Thu Jan 12 22:27:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cache local do Dataset

Como o dataset tem um tamanho grande e está no Google Drive distribuído em vários arquivos, queremos carregar todos em um único arquivo já tokenizado pelo *tokenizer* da GPT.

Para evitar problemas de sincronização, vamos copiar os arquivos para /content/tmp/, depois copiar de volta os arquivos resultantes.

In [ ]:
! mkdir -p "/content/tmp/"
! for i in {0..8}; do echo "model $i" && time cp -r /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/$i /content/tmp/; done

model 0

real	1m11.157s
user	0m0.016s
sys	0m2.026s
model 1

real	1m45.635s
user	0m0.029s
sys	0m1.899s
model 2
^C


In [ ]:

# ! ls "$PATH_DATA/kaggle/proc-1/$LABEL/" | wc -l
# ! ls "/content/tmp/$LABEL/" | wc -l

! for i in {0..8}; do echo "model $i" && du -sh "/content/tmp/$i" && ls "/content/tmp/$i" | wc -l; done

model 0
1.3G	/content/tmp/0
1541
model 1
989M	/content/tmp/1
2478
model 2
76M	/content/tmp/2
2942
model 3
16M	/content/tmp/3
475
model 4
1.8M	/content/tmp/4
42
model 5
67M	/content/tmp/5
751
model 6
28M	/content/tmp/6
398
model 7
141M	/content/tmp/7
1228
model 8
137M	/content/tmp/8
1013


In [ ]:
! cd $PATH_PROJECT && python main.py --cache --path="/content/tmp/"

Namespace(batch=None, cache=True, check=False, complete=False, epochs=None, label=None, limit=None, metrics=False, model=None, multi=False, path='/content/tmp/', process=False, test=False, train=False)
pid: 16787
Args: limit 1024
Args: epochs 2
Args: batch 160
Args: labels ['0', '1', '2', '3', '4', '5', '6', '7', '8']
Args: path /content/tmp/
Args: model_name gpt2
Args: multi False
Model: name gpt2
Model: label '0'
Model: num_labels 2
Model: downloading tokenizer...
Downloading: 100% 1.04M/1.04M [00:01<00:00, 748kB/s]
Downloading: 100% 456k/456k [00:01<00:00, 403kB/s]
Downloading: 100% 665/665 [00:00<00:00, 479kB/s]
Model: saving tokenizer...
Model: tokenizer saved!
Dataset: label 0 - eval - in model as torch.tensor(0)
Dataset: label 0 - eval - creating from /content/tmp/0 ...
Dataset: label 0 - eval - 154 files
Token indices sequence length is longer than the specified maximum sequence length for this model (95541 > 1024). Running this sequence through the model will result in indexin

In [ ]:
# copy files back
! time cp /content/tmp/*.csv "/content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/"


real	0m19.622s
user	0m0.001s
sys	0m0.028s


# Cache

In [ ]:
# cache dataset
! cd $PATH_PROJECT && python main.py --cache --model='gpt2' --limit='256'

Namespace(batch=None, cache=True, check=False, complete=False, epochs=None, fold=None, label=None, limit='256', metrics=False, model='gpt2', multi=False, path=None, process=False, test=False, train=False)
pid: 8146
Args: limit 256
Args: epochs 2
Args: batch 160
Args: labels ['0', '1', '2', '3', '4', '5', '6', '7', '8']
Args: path /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1
Args: model_name gpt2
Args: multi False
Args: fold 1
Model: name gpt2
Model: label '0'
Model: num_labels 2
Model: using saved tokenizer...
-----
label 0
-----
label 1
-----
label 2
-----
label 3
-----
label 4
-----
label 5
-----
label 6
-----
label 7
-----
label 8
0:00:00


# Treino

In [ ]:
! cd $PATH_PROJECT && python main.py --train --multi --model='gpt2' --limit='102400' --epochs='7' --batch='160' --fold='1'

Namespace(batch='160', cache=False, check=False, complete=False, epochs='7', fold='1', label=None, limit='102400', metrics=False, model='gpt2', multi=True, path=None, process=False, test=False, train=True)
pid: 1360
Args: limit 102400
Args: epochs 7
Args: batch 160
Args: labels ['all']
Args: path /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1
Args: model_name gpt2
Args: multi True
Args: fold 1
START all
Model: name gpt2
Model: label 'all'
Model: num_labels 9
Model: using saved tokenizer...
-----
label 0
-----
label 1
-----
label 2
-----
label 3
-----
label 4
-----
label 5
-----
label 6
-----
label 7
-----
label 8
Dataset: label 0 - train - in model as torch.tensor(0)
Dataset: label 0 - train - loading from /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/0.limit-102400.fold-1.chunk-32.train.csv ...
Dataset: label 0 - train - 3053845 chunks - 25.55 Gb
Dataset: label 1 - train - in model as torch.tensor(1)
Dataset: label 1 - train - loading from /content/drive/Shareddrives/GPTJ/d

# Test Binary

In [ ]:
! cd $PATH_PROJECT && python main.py --test --model='gpt2' --limit='1024' --batch='16'

Namespace(batch='16', cache=False, check=False, complete=False, epochs=None, label=None, limit='1024', metrics=False, model='gpt2', multi=False, path=None, process=False, test=True, train=False)
pid: 17025
Args: limit 1024
Args: epochs 2
Args: batch 16
Args: labels ['0', '1', '2', '3', '4', '5', '6', '7', '8']
Args: path /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1
Args: model_name gpt2
Args: multi False
Model: name gpt2
Model: label '0'
Model: num_labels 2
Model: using saved tokenizer...
Dataset: label 0 - test - in model as torch.tensor(0)
Dataset: label 0 - test - loading from /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/0.limit-1024.chunk-32.test.csv ...
Dataset: label 0 - test - 4922 chunks - 0.04 Gb
Dataset: label 1 - test - in model as torch.tensor(1)
Dataset: label 1 - test - loading from /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/1.limit-1024.chunk-32.test.csv ...
Dataset: label 1 - test - 7872 chunks - 0.06 Gb
Dataset: label 2 - test - in model as torch

In [ ]:
! cd $PATH_PROJECT && python main.py --metrics --model='gpt2' --limit='512' --batch='16'

# Test Multi

In [ ]:
! cd $PATH_PROJECT && python main.py --test --multi --model='gpt2' --limit='102400' --epochs='7' --batch='160' --fold='1'

python3: can't open file 'main.py': [Errno 2] No such file or directory


# Tensorboard

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir /content/drive/Shareddrives/GPTJ/data/gpt2/7.limit-256.chunk-32.epochs-5.batch-16/logs

# Temp

In [ ]:
from statistics import mean

l1 = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]
print(mean(l1), round(mean(l1)))

l2 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
print(mean(l2), round(mean(l2)))

In [ ]:
from collections import Counter

L = [1,   1, 3, 5, 6, 6, 6, 7]
W = [2.5, 2, 1, 1, 1, 1, 1, 1]

def WeightedCounter(L, W):
    c = {}
    for l, w in zip(L, W):
        if not l in c:
            c.update({l:0})
        c[l] += w
    return Counter(c)

c1 = Counter(L).most_common()[0][0]
c2 = WeightedCounter(L, W).most_common()[0][0]

print(c1)
print(c2)